In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


## Import desired packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import statsmodels.api as sm
import gc

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from tqdm import tqdm
import itertools
import gc


Mounted at /content/drive


In [ ]:
# Load images and labels
# We make sure it loads RGB things
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image = Image.open(os.path.join(folder_path, filename)).convert('RGB').resize((224, 224))
            images.append(np.array(image))
            labels.append(label)
    return images, labels

folder_0 = "/content/drive/MyDrive/data/non_hs"
folder_1 = "/content/drive/MyDrive/data/hs"

# Mark non_head_and_shoulder as 0 and head_and_shoulder as 1
images_0, labels_0 = load_images(folder_0, 0)
images_1, labels_1 = load_images(folder_1, 1)

images = np.array(images_0 + images_1)
labels = np.array(labels_0 + labels_1)

# Split data into train and test sets, we also stratify them 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rescale=1./255
)

datagen.fit(X_train)

### GoogLeNet

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
# Load the pretrained InceptionV3 model without the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
googlenet = tf.keras.Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pretrained model
for layer in base_model.layers:
    layer.trainable = False

# Compile model
googlenet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train GoogLeNet (InceptionV3) model
googlenet.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_test/255, y_test), epochs=10)
test_loss, test_accuracy = googlenet.evaluate(X_test / 255, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
33/33 [==============================] - 195s 6s/step - loss: 0.2427 - accuracy: 0.9194 - val_loss: 0.1845 - val_accuracy: 0.9583
Epoch 2/10
33/33 [==============================] - 188s 6s/step - loss: 0.1095 - accuracy: 0.9696 - val_loss: 0.1482 - val_accuracy: 0.9621
Epoch 3/10
33/33 [==============================] - 185s 6s/step - loss: 0.0951 - accuracy: 0.9725 - val_loss: 0.1331 - val_accuracy: 0.9621
Epoch 4/10
33/33 [==============================] - 185s 6s/step - loss: 0.0847 - accuracy: 0.9706 - val_loss: 0.1189 - val_accuracy: 0.9659
Epoch 5/10
33/33 [==============================] - 190s 6s/step - loss: 0.0801 - accuracy: 0.9734 - val_loss: 0.1201 - val_accuracy: 0.9659
Epoch 6/10
33/33 [==============================] - 187s 6s/step - loss: 0.0824 - accuracy: 0.9715 - val_loss: 0.1202 - val_accuracy: 0.9621
Epoch 7/10
33/33 [==============================] - 177s 5s/step - loss: 0.0696 - accuracy: 0.9763 - val_loss: 0.1122 - val_accuracy: 0.9659
Epoch 8/10
33